In [ ]:
pip install pydota

# Import

In [10]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import add
from keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook as tqdm

# Dataset

In [11]:
dataset_path = "cleaned_captions.txt"

In [12]:
text = open(dataset_path, 'rb').read().decode(encoding='utf-8')

In [13]:
text[:10]

'a child in'

### Data in each lines extracted 

In [14]:
sentences = text.splitlines()

In [ ]:
sentences

### Already cleaned and now is a list of sentences

### Making the vocabulary

In [16]:
def text_vocabulary(captions_list):
    
    # Initialize an empty set to store the vocabulary
    vocab = set()

    # Iterate over each caption in the list
    for caption in captions_list:
        # Split the caption into words and add them to the vocabulary set
        vocab.update(caption.split())

    return vocab

In [17]:
vocab = text_vocabulary(sentences)

In [18]:
len(vocab)

8828

In [ ]:
vocab

###  Adding the <#start> and <#end> tokens to the sentences

In [20]:
def add_start_end_tokens(sentences):

    sentences_with_tokens = []

    # Iterate over each sentence in the list
    for sentence in sentences:
        # Add start token '<start>' to the beginning of the sentence and end token '<end>' to the end
        sentence_with_tokens = '<start> ' + sentence + ' <end>'
        # Append the modified sentence to the list
        sentences_with_tokens.append(sentence_with_tokens)

    return sentences_with_tokens

In [21]:
# Call the add_start_end_tokens function with the sentences list
sentences_with_tokens = add_start_end_tokens(sentences)

In [22]:
# sentences_with_tokens = sentences

In [ ]:
sentences_with_tokens

# Tokenization

In [24]:
from keras.preprocessing.text import Tokenizer

In [25]:
def create_tokenizer(sentences_with_tokens):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences_with_tokens)
    return tokenizer

In [26]:
tokenizer = create_tokenizer(sentences_with_tokens)
dump(tokenizer, open('tokenizer.p', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
vocab_size 

8829

### Max length

In [27]:
#calculate maximum length of descriptions

def max_length(sentences_with_tokens):
    
    return max(len(d.split()) for d in sentences_with_tokens)

max_length = max_length(sentences_with_tokens)
max_length

38

In [28]:
tokenizer

# Converting list of sentencees into dictionary with key as numbers and values as group 5 sentences

In [29]:
def group_sentences_into_lists(sentences, group_size):
    
    grouped_lists = []
    for i in range(0, len(sentences), group_size):
        grouped_lists.append(sentences[i:i+group_size])
    return grouped_lists

def convert_to_dictionary(sentences, group_size):
    
    grouped_sentences = group_sentences_into_lists(sentences, group_size)
    dictionary = {}
    for i, group in enumerate(grouped_sentences, start=1):
        dictionary[i] = group
    return dictionary

In [ ]:
group_size = 5  # Define the group size

# Convert sentences to a dictionary
sentences_dictionary = convert_to_dictionary(sentences_with_tokens, group_size)

# Print the dictionary
for key, value in sentences_dictionary.items():
    print(f"{key}: {value}")

# Data generator part

In [31]:
def data_generator(descriptions, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():

            input_sequence, output_word = create_sequences(tokenizer, max_length, description_list)
            yield [[input_sequence], output_word]         

In [32]:
def create_sequences(tokenizer, max_length, desc_list):
    X1, y = list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            
            X1.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(y)

In [33]:
[a],b = next(data_generator(sentences_dictionary, tokenizer, max_length))
a.shape, b.shape

((58, 38), (58, 8829))

In [34]:
a

array([[   0,    0,    0, ...,    0,    0,    3],
       [   0,    0,    0, ...,    0,    3,    1],
       [   0,    0,    0, ...,    3,    1,   42],
       ...,
       [   0,    0,    0, ...,  314,   64,    1],
       [   0,    0,    0, ...,   64,    1,  194],
       [   0,    0,    0, ...,    1,  194, 2992]])

In [35]:
b

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

# Model

### MOdel 1 --- think need to train for more epoch tested only after 1 epoch

In [26]:
# from keras.utils import plot_model

# # define the captioning model
# def define_model(vocab_size, max_length):

#     # LSTM sequence model
#     inputs1 = Input(shape=(max_length,))
#     se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs1)
#     se2 = Dropout(0.5)(se1)
#     se3 = LSTM(256)(se2)

#     decoder2 = Dense(256, activation='relu')(se3)
#     outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
#     # tie it together [image, seq] [word]
#     model = Model(inputs=[inputs1], outputs=outputs)
#     model.compile(loss='categorical_crossentropy', optimizer='adam')
    
#     # summarize model
#     print(model.summary())
#     plot_model(model, to_file='model.png', show_shapes=True)
    
#     return model

### Modle 2 chatgpt improved

In [27]:
# def define_model(vocab_size, max_length):

#     # LSTM sequence model
#     inputs1 = Input(shape=(max_length,))
#     se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs1)
#     se2 = Dropout(0.5)(se1)
#     se3 = LSTM(1024, return_sequences=True)(se2)  # Adding return_sequences=True to return sequences for the next LSTM layer
#     se4 = Dropout(0.5)(se3)  # Adding dropout after the LSTM layer
#     se5 = LSTM(1024)(se4)  # Adding another LSTM layer
#     decoder1 = Dense(512, activation='relu')(se5)  # Adding a dense layer after the LSTM
#     decoder2 = Dense(512, activation='relu')(decoder1)
#     decoder3 = Dense(512, activation='relu')(decoder2)
#     outputs = Dense(vocab_size, activation='softmax')(decoder3)
    
#     # tie it together [image, seq] [word]
#     model = Model(inputs=[inputs1], outputs=outputs)
#     model.compile(loss='categorical_crossentropy', optimizer='adam')
    
#     # summarize model
#     print(model.summary())
#     # plot_model(model, to_file='model.png', show_shapes=True)
    
#     return model

# Final model

In [28]:
from keras.layers import Dropout

def define_model(vocab_size, max_length):

    # LSTM sequence model
    inputs1 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs1)
    se2 = Dropout(0.3)(se1)
    se3 = LSTM(1024, return_sequences=True)(se2)
    se4 = Dropout(0.3)(se3)
    se5 = LSTM(1024)(se4)
    decoder1 = Dense(512, activation='relu')(se5)
    dropout1 = Dropout(0.3)(decoder1)  # Adding dropout after the first dense layer
    decoder2 = Dense(512, activation='relu')(dropout1)
    dropout2 = Dropout(0.3)(decoder2)  # Adding dropout after the second dense layer
    decoder3 = Dense(512, activation='relu')(dropout2)
    dropout3 = Dropout(0.3)(decoder3)  # Adding dropout after the third dense layer
    outputs = Dense(vocab_size, activation='softmax')(dropout3)
    
    model = Model(inputs=[inputs1], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    print(model.summary())
    
    return model


In [29]:
# Define the model
model = define_model(vocab_size, max_length)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 38)]              0         
                                                                 
 embedding (Embedding)       (None, 38, 256)           2260224   
                                                                 
 dropout (Dropout)           (None, 38, 256)           0         
                                                                 
 lstm (LSTM)                 (None, 38, 1024)          5246976   
                                                                 
 dropout_1 (Dropout)         (None, 38, 1024)          0         
                                                                 
 lstm_1 (LSTM)               (None, 1024)              8392704   
                                                                 
 dense (Dense)               (None, 512)               524800

# Train the model

In [31]:
# train our model
print('Descriptions: train=', len(sentences_dictionary))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)

model = define_model(vocab_size, max_length)
epochs = 10
steps = len(sentences_dictionary)
# making a directory models to save our models
# os.mkdir("models")
for i in range(epochs):
    generator = data_generator(sentences_dictionary, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch= steps, verbose=1)
    model.save("models/textmodel_" + str(i) + ".h5")

Descriptions: train= 8092
Vocabulary Size: 8829
Description Length:  38
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 38)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 38, 256)           2260224   
                                                                 
 dropout_10 (Dropout)        (None, 38, 256)           0         
                                                                 
 lstm_4 (LSTM)               (None, 38, 1024)          5246976   
                                                                 
 dropout_11 (Dropout)        (None, 38, 1024)          0         
                                                                 
 lstm_5 (LSTM)               (None, 1024)              8392704   
                                                     

C:\Users\abhij\AppData\Local\Temp\ipykernel_17644\826986174.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator, epochs=1, steps_per_epoch= steps, verbose=1)


8092/8092 [==============================] - 3450s 426ms/step - loss: 3.3765


In [32]:
model.save("WordLevelTextGenModel10.h5")

# Generating text

In [2]:
# input_text = "a girl"
predict_next_words= 30
# # model = load_model("WordLevelTextGenModel.h5")

# for _ in range(predict_next_words):
#     token_list = tokenizer.texts_to_sequences([input_text])[0]
#     print(token_list)
#     token_list = pad_sequences([token_list], maxlen=max_length, padding='pre')
#     predicted = np.argmax(model.predict(token_list), axis=-1)
#     output_word = ""
#     for word, index in tokenizer.word_index.items():
#         if index == predicted:
#             output_word = word
#             break
#     input_text += " " + output_word

# print(input_text)

In [ ]:
input_text = "a man"

for _ in range(predict_next_words):
    token_list = tokenizer.texts_to_sequences([input_text])[0]
    print(token_list)
    token_list = pad_sequences([token_list], maxlen=max_length, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            if word == "end":  # Check if the predicted word is the end token
                break  # Stop text generation if the end token is predicted
            output_word = word
            break
    input_text += " " + output_word

print(input_text)

# Bleu score

In [ ]:
pip install datasets

In [6]:
from datasets import load_metric

bleu = load_metric("bleu")
predictions = [["two", "dogs", "are", "running", "through","the", "water"]]
references = [
    [["Two", "dogs", "are", "running", "through","the", "water"]]
]

bleu.compute(predictions=predictions, references=references)

C:\Users\abhij\miniconda3\envs\tensorflow2.10\lib\site-packages\datasets\load.py:753: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'bleu': 0.8091067115702212,
 'precisions': [0.8571428571428571, 0.8333333333333334, 0.8, 0.75],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 7,
 'reference_length': 7}

In [7]:
# Data
precisions = [0.8571428571428571, 0.8333333333333334, 0.8, 0.75]
brevity_penalty = 1.0

# BLEU-1
bleu_1 = precisions[0]

# BLEU-2
bleu_2 = (precisions[0] * precisions[1]) ** 0.5

# BLEU-3
bleu_3 = (precisions[0] * precisions[1] * precisions[2]) ** (1/3)

# BLEU-4
bleu_4 = (precisions[0] * precisions[1] * precisions[2] * precisions[3]) ** 0.25

print("BLEU-1 :", bleu_1)
print("BLEU-2 :", bleu_2)
print("BLEU-3 :", bleu_3)
print("BLEU-4 :", bleu_4)


BLEU-1 : 0.8571428571428571
BLEU-2 : 0.8451542547285166
BLEU-3 : 0.8298265333662435
BLEU-4 : 0.8091067115702212
